In [ ]:
from torch import load
from data.DataLoad_utils import mesh_resolution,cut_type
from construct_model import classifier
from torch import no_grad
from test_util import test
from training_utils import test_DataLoader
from sklearn.metrics import roc_curve, auc
import os
import pickle
import matplotlib.pyplot as plt

In [ ]:
checkpoint = load(f'./checkpoints/best_model_00{mesh_resolution}_{cut_type}.pth')
classifier.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
vote_num=3

with no_grad():
  if vote_num>1:
    instance_acc, class_acc, predictions, labels = test(classifier.eval(), test_DataLoader, vote_num=vote_num, num_class=2)
  elif 1==vote_num:
    instance_acc, class_acc, predictions, labels, all_features = test(classifier.eval(), test_DataLoader, vote_num=vote_num, num_class=2)
    print('Test Instance Accuracy: %f, Class Accuracy: %f' % (instance_acc, class_acc))

In [ ]:
labels.shape

In [ ]:
predictions.shape

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()

class_idx=1#只关心正例（破裂了）
fpr[class_idx], tpr[class_idx], _ = roc_curve(labels == class_idx, predictions[:, class_idx])
roc_auc[class_idx] = auc(fpr[class_idx], tpr[class_idx])

roc_data=[(fpr, tpr, roc_auc)]

os.makedirs(name='./results', exist_ok=True)
with open(f'./results/PC_NN_roc_data_00{mesh_resolution}_{cut_type}.pkl', 'wb') as f:
        pickle.dump((fpr, tpr, roc_auc), f)

In [ ]:
curves_to_merge = ['./results/PC_NN_roc_data_005_ninja.pkl']  # 你需要指定所有保存的文件名,'./results/PC_NN_roc_data_005_ninja.pkl'

plt.figure()

for curve in curves_to_merge:
    with open(curve, 'rb') as f:
        fpr, tpr, roc_auc = pickle.load(f)
    class_idx=1
    curve=curve.split('.pkl')[0]
    curve=curve.split('_')[-1]
    plt.plot(fpr[class_idx], tpr[class_idx], label=f'{curve}, Class {class_idx} (AUC = {roc_auc[class_idx]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()